In [3]:
import os
import nltk
import string
import re
import numpy as np

parser_dir = "../brain_syntactic_representations/incremental-top-down-parser"

save_dir = "features"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [4]:
def get_surprisal(row):
    all_num = re.findall(r"[-+]?\d*\.\d+|\d+", row)
    print(all_num)
    return float(all_num[-7])

In [1]:
all_text = []
hp_text = []
para = ""

In [2]:
narrative_text = open("./stimuli/text/lw1new.txt","r").readlines()
for eachsent in narrative_text[0].split('.'):
    all_text.append(eachsent.strip()+'.')
    for j in range(len(eachsent.strip().split())):
        #if eachsent.strip().split()[j].strip() not in [',','?','!','-','"',';']:
        hp_text.append(eachsent.strip().split()[j].strip())
print(all_text)

narrative_text = open("./stimuli/text/cable_spool_fort_new.txt","r").readlines()
for eachsent in narrative_text[0].split('.'):
    all_text.append(eachsent.strip()+'.')
    for j in range(len(eachsent.strip().split())):
        #if eachsent.strip().split()[j].strip() not in [',','?','!','-','"',';']:
        hp_text.append(eachsent.strip().split()[j].strip())
        
narrative_text = open("./stimuli/text/easy_money_new.txt","r").readlines()
for eachsent in narrative_text[0].split('.'):
    all_text.append(eachsent.strip()+'.')
    for j in range(len(eachsent.strip().split())):
        #if eachsent.strip().split()[j].strip() not in [',','?','!','-','"',';']:
        hp_text.append(eachsent.strip().split()[j].strip())
        
narrative_text = open("./stimuli/text/black_willow_new.txt","r").readlines()
for eachsent in narrative_text[0].split('.'):
    all_text.append(eachsent.strip()+'.')
    for j in range(len(eachsent.strip().split())):
        #if eachsent.strip().split()[j].strip() not in [',','?','!','-','"',';']:
        hp_text.append(eachsent.strip().split()[j].strip())

['Tara stood stock still waiting for the first tiny gleam from the scout craft to appear in the darkness of the.', 'The gentle constant breeze of recycled air from the vent above blew an annoying hair against her nose but she ignored it.', 'A gasp from the psychic broke her silent vigil and she turned.', 'Results Harmon she suppressed the surge of annoyance that ran through her as she contemplated the gift of getting all the hot news first.', 'face slowly animated joy sweeping in to replace stern concentration.', 'says the planet tiny away contemplated to not blonde slowly broke by Thriving with life large and small forms no buildings of any kind metals and stable atmosphere excellent onto the only head planet pointed blackness now get gentle imposing alpha.', 'He says we should come now immediately before the probe is reported late Harmon that good nothing son of beta get his the gate and here.', 'NOW or open fire on him when we come Tara Of all the no doubt with orders from Mason was

In [5]:
ptr = 0
all_surprisal = []
for i in range(len(hp_text)):
    all_surprisal.append([])

len_remaining = len(hp_text[ptr])
for para in all_text:
    all_sents = nltk.sent_tokenize(para)
    for sent_number in range(len(all_sents)):
        sent = all_sents[sent_number]
        if sent == '..."': # useless sentence, needs to be skipped
            print('Encountered ..." skipping it')
            ptr += 1
            while(ptr < len(hp_text) and hp_text[ptr] == "+"):
                ptr += 1
            len_remaining = len(hp_text[ptr])
            continue
        sent = sent.translate(str.maketrans({key: " {0} ".format(key) for key in string.punctuation})).strip()
        inp_f = open("input.txt", "w+")
        inp_f.write(sent)
        inp_f.close()
        os.system("{}/bin/tdparse -p -F output.txt {}/parse.model.wsj.slc.p05 input.txt".format(parser_dir, parser_dir))
        
        output = open("output.txt", "r").readlines()
        print(sent)
        
        tokens_ori = sent.split(" ")
        tokens = []
        for j in range(len(tokens_ori)): # remove whitespace tokens
            if len(tokens_ori[j].strip()) != 0:
                tokens.append(tokens_ori[j])
        
        k = 0
        for j in range(len(tokens)):
            len_remaining -= len(tokens[j])
            if len_remaining == 0:
                if hp_text[ptr] == '"':
                    ptr += 1
                    len_remaining = len(hp_text[ptr])
                    continue
                print("Word completed by {} len remaining = {}".format(tokens[j], len_remaining))
                
                row = output[k+1]
                surprisal = get_surprisal(row)
                all_surprisal[ptr].append(surprisal)
                
                k += 1
                
                print("Word = {}".format(hp_text[ptr]))
                print("Row = {}".format(row))
                print("All surprisal = {}".format(all_surprisal[ptr]))
                
                ptr += 1
                while(ptr < len(hp_text) and hp_text[ptr] == "+"):
                    ptr += 1
                if ptr < len(hp_text):
                    len_remaining = len(hp_text[ptr])
            else:
                print("Skipped {} len remaining = {}".format(tokens[j], len_remaining))
                if tokens[j] != '"' and ((j == len(tokens) - 1) or (tokens[j+1] != '"')):
                    row = output[k+1]
                    surprisal = get_surprisal(row)
                    all_surprisal[ptr].append(surprisal)
                    k += 1
        print()

Tara stood stock still waiting for the first tiny gleam from the scout craft to appear in the darkness of the .
Word completed by Tara len remaining = 0
['1', '5.847', '5.847', '1.062', '4.785', '1.560', '1.92', '1.00', '0.54', '3.9']
Word = Tara
Row = pfix:1           Tara   5.847  5.847 1.062  4.785  1.560  1.92  1.00  0.54  3.9

All surprisal = [1.062]
Word completed by stood len remaining = 0
['2', '15.873', '10.025', '2.782', '7.243', '0.795', '2.00', '1.02', '1.02', '3.0']
Word = stood
Row = pfix:2          stood  15.873 10.025 2.782  7.243  0.795  2.00  1.02  1.02  3.0

All surprisal = [2.782]
Word completed by stock len remaining = 0
['3', '25.693', '9.820', '4.907', '4.913', '1.585', '3.50', '0.99', '0.73', '2.5']
Word = stock
Row = pfix:3          stock  25.693  9.820 4.907  4.913  1.585  3.50  0.99  0.73  2.5

All surprisal = [4.907]
Word completed by still len remaining = 0
['4', '33.852', '8.159', '4.715', '3.444', '1.919', '3.51', '1.17', '1.01', '3.1']
Word = still
Row =

A gasp from the psychic broke her silent vigil and she turned .
Skipped A len remaining = -99
['1', '3.993', '3.993', '0.843', '3.150', '0.279', '2.05', '1.00', '0.93', '4.1']
Skipped gasp len remaining = -103
['2', '24.771', '20.778', '10.189', '10.589', '0.350', '2.15', '1.07', '1.00', '3.0']
Skipped from len remaining = -107
['3', '30.398', '5.627', '2.172', '3.456', '0.469', '3.18', '0.99', '0.98', '2.0']
Skipped the len remaining = -110
['4', '31.611', '1.214', '0.869', '0.345', '0.381', '4.16', '1.02', '1.02', '2.0']
Skipped psychic len remaining = -117
['5', '47.712', '16.101', '1.454', '14.647', '0.641', '4.21', '1.00', '0.95', '1.1']
Skipped broke len remaining = -122
['6', '64.561', '16.849', '7.459', '9.390', '1.988', '6.20', '0.65', '0.35', '3.9']
Skipped her len remaining = -125
['7', '71.583', '7.022', '3.690', '3.332', '2.229', '8.43', '1.97', '1.16', '2.1']
Skipped silent len remaining = -131
['8', '85.762', '14.180', '2.715', '11.465', '2.762', '8.36', '0.63', '0.00', 

He says we should come now immediately before the probe is reported late Harmon that good nothing son of beta get his the gate and here .
Skipped He len remaining = -532
['1', '3.827', '3.827', '2.389', '1.437', '0.005', '2.00', '1.00', '1.00', '4.0']
Skipped says len remaining = -536
['2', '7.319', '3.493', '1.464', '2.029', '0.005', '2.00', '1.00', '1.00', '3.0']
Skipped we len remaining = -538
['3', '11.985', '4.666', '2.027', '2.638', '0.064', '4.99', '1.00', '0.99', '4.0']
Skipped should len remaining = -544
['4', '16.330', '4.345', '1.445', '2.900', '0.006', '5.00', '1.01', '1.01', '3.0']
Skipped come len remaining = -548
['5', '21.442', '5.112', '0.179', '4.933', '0.028', '6.00', '1.00', '1.00', '2.0']
Skipped now len remaining = -551
['6', '26.903', '5.461', '1.867', '3.594', '0.186', '7.02', '1.00', '0.98', '2.0']
Skipped immediately len remaining = -562
['7', '34.451', '7.549', '2.249', '5.300', '0.669', '7.04', '1.02', '0.82', '2.7']
Skipped before len remaining = -568
['8',

Mahogany black curly hair cut short in the typical military style framed a dark skinned face .
Skipped Mahogany len remaining = -841
['1', '14.872', '14.872', '3.147', '11.725', '0.551', '1.89', '1.00', '0.85', '3.9']
Skipped black len remaining = -846
['2', '26.136', '11.264', '2.055', '9.209', '1.324', '2.30', '1.02', '0.84', '1.5']
Skipped curly len remaining = -851
['3', '34.840', '8.704', '1.097', '7.607', '1.856', '2.36', '1.05', '0.84', '2.8']
Skipped hair len remaining = -855
['4', '47.523', '12.682', '3.572', '9.110', '1.182', '2.36', '0.00', '0.00', '1.1']
Skipped cut len remaining = -858
['5', '55.470', '7.947', '1.298', '6.649', '1.533', '2.36', '1.16', '0.70', '1.7']
Skipped short len remaining = -863
['6', '64.508', '9.038', '2.473', '6.565', '2.508', '4.00', '0.11', '0.00', '2.6']
Skipped in len remaining = -865
['7', '68.590', '4.083', '1.695', '2.388', '2.386', '3.80', '0.16', '0.00', '3.3']
Skipped the len remaining = -868
['8', '69.986', '1.396', '1.055', '0.341', '2

Her expression completed the imposing effect .
Skipped Her len remaining = -1145
['1', '7.338', '7.338', '2.290', '5.049', '0.490', '1.94', '1.00', '0.88', '3.9']
Skipped expression len remaining = -1155
['2', '19.262', '11.923', '0.576', '11.348', '0.465', '1.96', '1.04', '1.01', '1.0']
Skipped completed len remaining = -1164
['3', '27.698', '8.436', '1.637', '6.799', '0.425', '2.10', '1.11', '1.02', '3.0']
Skipped the len remaining = -1167
['4', '29.355', '1.657', '1.218', '0.439', '0.217', '3.08', '1.10', '1.06', '2.1']
Skipped imposing len remaining = -1175
['5', '44.645', '15.290', '4.972', '10.318', '0.636', '3.23', '0.99', '0.92', '1.2']
Skipped effect len remaining = -1181
['6', '51.712', '7.067', '0.324', '6.743', '0.205', '3.07', '1.10', '1.10', '1.0']
Skipped . len remaining = -1182
['7', '53.486', '1.775', '1.774', '0.001', '0.028', '1.00', '1.03', '1.03', '3.0']

She picked imaginary flecks off her stark gray jumpsuit and snorted .
Skipped She len remaining = -1185
['1', '

...

Indeed if still could could at any time follow advice seek a profession more grounded in reality .
Skipped Indeed len remaining = -36356
['1', '6.496', '6.496', '2.963', '3.533', '0.135', '2.00', '1.00', '0.98', '4.0']
Skipped if len remaining = -36358
['2', '15.138', '8.642', '4.375', '4.267', '1.197', '2.11', '0.48', '0.00', '2.4']
Skipped still len remaining = -36363
['3', '24.449', '9.312', '2.759', '6.552', '3.018', '4.31', '1.02', '0.40', '3.2']
Skipped could len remaining = -36368
['4', '30.450', '6.001', '3.765', '2.236', '1.231', '4.01', '2.96', '2.96', '3.0']
Skipped could len remaining = -36373
['5', '43.159', '12.709', '10.643', '2.066', '1.859', '4.85', '0.93', '0.83', '2.3']
Skipped at len remaining = -36375
['6', '51.201', '8.041', '6.441', '1.601', '2.803', '6.03', '0.95', '0.62', '2.2']
Skipped any len remaining = -36378
['7', '58.576', '7.376', '1.722', '5.654', '2.274', '7.17', '0.07', '0.00', '2.0']
Skipped time len remaining = -36382
['8', '63.930', '5.353', '0.936

As the stream wound its way through the largest pines grown so great as to eclipse that feature of the landscape they had once crowded about for sweet water as Allan continued slowing his pace to climb across the toes of the giants pale perfectly take listen seek nemesis desirable giants owners ink told off .
Skipped As len remaining = -36904
['1', '4.778', '4.778', '1.671', '3.106', '1.106', '2.01', '1.00', '0.49', '4.0']
Skipped the len remaining = -36907
['2', '6.628', '1.850', '1.151', '0.699', '0.785', '3.68', '0.63', '0.00', '2.7']
Skipped stream len remaining = -36913
['3', '16.569', '9.941', '0.524', '9.417', '0.732', '3.74', '1.09', '1.09', '1.0']
Skipped wound len remaining = -36918
['4', '27.629', '11.060', '1.374', '9.686', '0.441', '4.00', '1.38', '1.23', '3.0']
Skipped its len remaining = -36921
['5', '32.210', '4.581', '3.862', '0.719', '0.981', '5.24', '0.92', '0.76', '2.2']
Skipped way len remaining = -36924
['6', '38.096', '5.885', '0.758', '5.127', '0.956', '5.22', '

Emerging from between the trunks Allan set his feet softly down onto the carpet of needles and stopped .
Skipped Emerging len remaining = -37254
['1', '10.484', '10.484', '1.389', '9.095', '0.620', '2.00', '1.00', '0.71', '4.0']
Skipped from len remaining = -37258
['2', '17.559', '7.075', '3.522', '3.553', '0.710', '3.99', '0.53', '0.00', '3.0']
Skipped between len remaining = -37265
['3', '27.300', '9.741', '5.271', '4.470', '1.397', '5.86', '0.94', '0.80', '2.9']
Skipped the len remaining = -37268
['4', '30.770', '3.470', '3.258', '0.212', '1.324', '5.93', '0.00', '0.00', '2.0']
Skipped trunks len remaining = -37274
['5', '47.283', '16.514', '2.576', '13.937', '1.105', '5.98', '1.06', '1.06', '1.0']
Skipped Allan len remaining = -37279
['6', '62.560', '15.277', '5.667', '9.610', '2.419', '6.57', '0.50', '0.00', '4.4']
Skipped set len remaining = -37282
['7', '71.648', '9.087', '2.645', '6.443', '1.930', '7.15', '0.05', '0.00', '3.0']
Skipped his len remaining = -37285
['8', '76.840',

Though the elders trunks fell and died their children first roots grown from the very branches that now lay dead continued a continuous line from the most ancient of their ancestors a loosely fingers denied all own screaming critical civilized rounding lowering themselves together into bathed great elder cultured .
Skipped Though len remaining = -37552
['1', '6.543', '6.543', '1.115', '5.428', '0.595', '2.00', '1.00', '0.78', '4.0']
Skipped the len remaining = -37555
['2', '8.162', '1.619', '1.301', '0.318', '0.289', '3.98', '1.21', '1.21', '3.0']
Skipped elders len remaining = -37561
['3', '24.387', '16.225', '2.298', '13.927', '0.319', '4.00', '1.01', '1.00', '1.0']
Skipped trunks len remaining = -37567
['4', '41.321', '16.934', '4.668', '12.267', '0.822', '4.02', '1.00', '0.82', '1.4']
Skipped fell len remaining = -37571
['5', '46.900', '5.578', '1.669', '3.909', '0.110', '4.03', '1.29', '1.29', '3.0']
Skipped and len remaining = -37574
['6', '53.949', '7.050', '6.795', '0.255', '1.

Collapsing at its base Allan wrapped his arms around the stoic tree and let forth a moan a cry of purest agony that escaped him as the first tears seeped from the corners of his eyes and slid down his cheeks falling to the ground and seeping though the fallen leaves and needles to join the water of the stream flowing through the ground beneath them .
Skipped Collapsing len remaining = -37970
['1', '13.169', '13.169', '4.389', '8.779', '0.788', '2.03', '1.00', '0.81', '4.0']
Skipped at len remaining = -37972
['2', '17.567', '4.399', '1.647', '2.752', '0.538', '3.04', '1.16', '1.11', '2.0']
Skipped its len remaining = -37975
['3', '21.207', '3.640', '3.172', '0.468', '0.355', '4.02', '1.04', '1.04', '2.0']
Skipped base len remaining = -37979
['4', '30.725', '9.518', '0.158', '9.360', '0.798', '4.02', '1.00', '0.83', '1.0']
Skipped Allan len remaining = -37984
['5', '44.651', '13.926', '3.722', '10.203', '1.687', '3.52', '0.84', '0.51', '2.7']
Skipped wrapped len remaining = -37991
['6', 

In [6]:
len(all_surprisal)

8561

In [7]:
mean_surprisal = []
for i in range(len(all_surprisal)):
    print(hp_text[i])
    print(all_surprisal[i])
    if len(all_surprisal[i]) != 0:
        mean_surprisal.append(np.mean(all_surprisal[i]))
    else:
        mean_surprisal.append(0)
    print(mean_surprisal[-1])

Tara
[1.062]
1.062
stood
[2.782]
2.782
stock
[4.907]
4.907
still
[4.715]
4.715
waiting
[4.737]
4.737
for
[1.838]
1.838
the
[0.937]
0.937
first
[1.632]
1.632
tiny
[3.44]
3.44
gleam
[0.358]
0.358
from
[1.31]
1.31
the
[1.085]
1.085
scout
[0.827]
0.827
craft
[1.931]
1.931
to
[3.141]
3.141
appear
[1.376]
1.376
in
[1.131]
1.131
the
[0.9]
0.9
darkness
[0.245]
0.245
of
[1.736]
1.736
the
[1.135]
1.135
The
[7.393, 0.878, 1.783, 2.234, 0.507, 1.773, 2.228, 0.864, 2.192, 1.061, 12.163, 1.652, 7.72, 1.537, 1.013, 0.87, 0.898, 2.974, 0.961, 2.22, 3.487, 1.445, 3.164, 3.038, 0.843, 10.189, 2.172, 0.869, 1.454, 7.459, 3.69, 2.715, 0.55, 3.487, 2.884, 1.254, 3.529, 3.09, 2.156, 4.612, 2.383, 2.649, 0.682, 0.942, 0.728, 2.628, 4.094, 1.54, 3.358, 2.741, 2.73, 1.376, 2.209, 0.669, 1.067, 3.56, 1.241, 1.649, 1.781, 0.513, 2.953, 7.275, 1.24, 3.594, 3.291, 1.013, 4.238, 1.88, 6.599, 1.183, 2.29, 3.383, 3.578, 6.467, 1.224, 0.483, 3.07, 6.84, 3.485, 3.532, 4.477, 3.15, 4.673, 2.911, 1.816, 2.457, 2.9, 2.096

[]
0
out
[]
0
as
[]
0
he
[]
0
stood
[]
0
back
[]
0
up
[]
0
Something
[]
0
as
[]
0
it
[]
0
spun
[]
0
past
[]
0
Chad
[]
0
turned
[]
0
to
[]
0
look
[]
0
inside
[]
0
never
[]
0
did
[]
0
stop
[]
0
flat
[]
0
dinner
[]
0
fun
[]
0
Tucker
[]
0
there
[]
0
with
[]
0
surprised
[]
0
expression
[]
0
crossed
[]
0
eyes
[]
0
the
[]
0
in
[]
0
one
[]
0
hand
[]
0
the
[]
0
screwdriver
[]
0
held
[]
0
up
[]
0
high
[]
0
in
[]
0
the
[]
0
other
[]
0
ready
[]
0
to
[]
0
stab
[]
0
down
[]
0
into
[]
0
back
[]
0
flat
[]
0
flint
[]
0
rock
[]
0
split
[]
0
forehead
[]
0
was
[]
0
buried
[]
0
deep
[]
0
solid
[]
0
like
[]
0
part
[]
0
of
[]
0
Halloween
[]
0
costume
[]
0
complete
[]
0
with
[]
0
drop
[]
0
of
[]
0
blood
[]
0
starting
[]
0
down
[]
0
toward
[]
0
his
[]
0
nose
[]
0
Tucker
[]
0
dropped
[]
0
down
[]
0
like
[]
0
jacket
[]
0
from
[]
0
hook
[]
0
lay
[]
0
folded
[]
0
over
[]
0
his
[]
0
own
[]
0
lap
[]
0
in
[]
0
way
[]
0
that
[]
0
made
[]
0
Chad
[]
0
think
[]
0
of
[]
0
broken
[]
0
things
[]
0
You
[]
0
ok
[]
0
It
[]
0
w

0
carved
[]
0
later
[]
0
not
[]
0
be
[]
0
whoever
[]
0
built
[]
0
it
[]
0
The
[]
0
markers
[]
0
are
[]
0
places
[]
0
where
[]
0
safe
[]
0
to
[]
0
come
[]
0
here
[]
0
and
[]
0
leave
[]
0
ignored
[]
0
id
[]
0
him
[]
0
pages
[]
0
someone
[]
0
several
[]
0
took
[]
0
tell
[]
0
yes
[]
0
some
[]
0
id
[]
0
pattern
[]
0
Safe
[]
0
I
[]
0
looked
[]
0
around
[]
0
One
[]
0
thing
[]
0
ot
[]
0
this
[]
0
place
[]
0
seem
[]
0
was
[]
0
safe
[]
0
there
[]
0
sounded
[]
0
slipped
[]
0
told
[]
0
mentioned
[]
0
for
[]
0
device
[]
0
issued
[]
0
all
[]
0
kentucky
[]
0
traffic
[]
0
whoever
[]
0
voice
[]
0
pushing
[]
0
Let
[]
0
me
[]
0
show
[]
0
you
[]
0
the
[]
0
rest
[]
0
He
[]
0
started
[]
0
walking
[]
0
down
[]
0
the
[]
0
steps
[]
0
and
[]
0
around
[]
0
a
[]
0
corner
[]
0
It
[]
0
would
[]
0
have
[]
0
taken
[]
0
plenty
[]
0
to
[]
0
surprise
[]
0
me
[]
0
after
[]
0
what
[]
0
I
[]
0
had
[]
0
already
[]
0
seen
[]
0
but
[]
0
My
[]
0
jaw
[]
0
was
[]
0
banging
[]
0
against
[]
0
my
[]
0
shoelaces
[]
0
when
[]
0
I
[]


hand
[]
0
and
[]
0
it
[]
0
at
[]
0
the
[]
0
window
[]
0
Damn
[]
0
you
[]
0
he
[]
0
cried
[]
0
will
[]
0
you
[]
0
never
[]
0
leave
[]
0
me
[]
0
be
[]
0
Shall
[]
0
I
[]
0
never
[]
0
finish
[]
0
a
[]
0
story
[]
0
in
[]
0
peace
[]
0
He
[]
0
pounded
[]
0
his
[]
0
desk
[]
0
in
[]
0
vain
[]
0
cursing
[]
0
the
[]
0
foul
[]
0
that
[]
0
drove
[]
0
him
[]
0
The
[]
0
next
[]
0
day
[]
0
Allan
[]
0
sat
[]
0
on
[]
0
a
[]
0
patterned
[]
0
chair
[]
0
in
[]
0
a
[]
0
well
[]
0
furnished
[]
0
parlor
[]
0
Taking
[]
0
a
[]
0
measured
[]
0
sip
[]
0
of
[]
0
white
[]
0
wine
[]
0
from
[]
0
his
[]
0
crystal
[]
0
glass
[]
0
he
[]
0
glanced
[]
0
at
[]
0
his
[]
0
host
[]
0
a
[]
0
cultured
[]
0
man
[]
0
dressed
[]
0
in
[]
0
a
[]
0
light
[]
0
suit
[]
0
of
[]
0
the
[]
0
current
[]
0
fashion
[]
0
and
[]
0
wearing
[]
0
his
[]
0
pale
[]
0
hair
[]
0
with
[]
0
a
[]
0
heavy
[]
0
set
[]
0
of
[]
0
who
[]
0
passed
[]
0
him
[]
0
a
[]
0
silver
[]
0
tray
[]
0
Cucumber
[]
0
sandwich
[]
0
Allan
[]
0
took
[]
0
a
[]
0
sandwich
[]
0
b

[]
0
the
[]
0
period
[]
0
At
[]
0
long
[]
0
last
[]
0
Nathan
[]
0
reached
[]
0
the
[]
0
end
[]
0
and
[]
0
set
[]
0
down
[]
0
his
[]
0
reading
[]
0
on
[]
0
the
[]
0
table
[]
0
between
[]
0
them
[]
0
Allan
[]
0
leaned
[]
0
forward
[]
0
unconsciously
[]
0
unhappiness
[]
0
way
[]
0
days
[]
0
must
[]
0
can
[]
0
at
[]
0
interrogation
[]
0
imagine
[]
0
imagined
[]
0
encouragement
[]
0
lines
[]
0
delicately
[]
0
characters
[]
0
abandoning
[]
0
outline
[]
0
this
[]
0
the
[]
0
best
[]
0
story
[]
0
ever
[]
0
written
[]
0
Allan
[]
0
exhaled
[]
0
and
[]
0
leaned
[]
0
back
[]
0
into
[]
0
the
[]
0
chair
[]
0
his
[]
0
face
[]
0
relaxing
[]
0
in
[]
0
imitation
[]
0
of
[]
0
his
[]
0
thoughts
[]
0
So
[]
0
he
[]
0
asked
[]
0
you
[]
0
think
[]
0
a
[]
0
waste
[]
0
of
[]
0
ink
[]
0
and
[]
0
paper
[]
0
a
[]
0
futile
[]
0
expedition
[]
0
into
[]
0
or
[]
0
literary
[]
0
debauchery
[]
0
Heavens
[]
0
no
[]
0
said
[]
0
Nathan
[]
0
aghast
[]
0
This
[]
0
is
[]
0
one
[]
0
of
[]
0
the
[]
0
strongest
[]
0
works
[]
0
re

In [8]:
np.save(os.path.join(save_dir,"syntactic_surprisal"), mean_surprisal)

In [12]:
#stories = ['lw1','cable','easymoney','balckwillow']
story_len = [0,668,1503,2637,3753]
syntactic_surprisal = []
for i in np.arange(1,5):
    #aa = np.load("./features/pos_tags_"+eachstory+".npy", allow_pickle=True)
    syntactic_surprisal.append(mean_surprisal[story_len[i-1]:story_len[i-1]+story_len[i]])

In [10]:
np.array(mean_surprisal).shape

(8561,)

In [13]:
np.save('syntactic_surprisal_allstories', syntactic_surprisal)

/home/subbainria/anaconda3/lib/python3.8/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
